This Python notebook merges the Overture data with the OSM data for ground truth verification purposes.

In [ ]:
!pip install pandas --quiet
!pip install geopandas --quiet
!pip install shapely --quiet
!pip install duckdb --quiet
!pip install jupysql --quiet
!pip install duckdb-engine --quiet
!pip install folium matplotlib mapclassify --quiet
!pip install pyproj --quiet

In [ ]:
import pandas as pd
import geopandas as gpd
import duckdb
from shapely import wkt
from shapely import Point
from shapely.ops import nearest_points
import shapely
from pyproj import Transformer

In [ ]:
%reload_ext sql

In [ ]:
%sql duckdb://

In [ ]:
%sql INSTALL spatial;
%sql INSTALL httpfs;
%sql LOAD spatial;
%sql LOAD httpfs;
%sql SET s3_region='us-west-2'

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
places = gpd.read_parquet(path='santa_cruz_places_building_snap.parquet')

In [ ]:
gers_ids_list = str(list(places['id'])).replace('[', '(').replace(']', ')')

In [ ]:
%%sql
COPY(
    SELECT
        id,
        tags->>'name' AS name,
        lat,
        lon,
        tags->>'place' AS place
    FROM st_readosm('norcal-latest.osm.pbf')
    WHERE lon BETWEEN -122.09 AND -121.91 AND
          lat BETWEEN 36.93 AND 37.02
) TO 'osm_nodes.parquet'

In [ ]:
%%sql
COPY(
WITH ways AS (
    SELECT
        id,
        kind,
        tags ->> 'name' AS name,
        refs
    FROM st_readosm('norcal-latest.osm.pbf')
    WHERE kind = 'way'
      AND name IS NOT NULL
      AND array_length(refs) > 0
),
nodes AS (
    SELECT
        id
    FROM 'osm_nodes.parquet'
)
SELECT
    ways.name,
    ways.refs
FROM ways
JOIN nodes ON nodes.id = ways.refs[1]
) TO 'osm_ways.parquet'

In [ ]:
ways = pd.read_parquet(path='osm_ways.parquet')

In [ ]:
nodes = pd.read_parquet(path='osm_nodes.parquet')

In [ ]:
ways['coords'] = ways.refs.apply(lambda x: [(nodes[nodes['id'] == v]['lat'].values[0], nodes[nodes['id'] == v]['lon'].values[0]) if len(nodes[nodes['id'] == v]['lat'].values) > 0 else None for v in x])

In [ ]:
ways['coords'] = ways.coords.apply(lambda x: None if None in x else x)

In [ ]:
ways = ways.dropna()

In [ ]:
ways['average_coord'] = ways.coords.apply(lambda x: shapely.centroid(shapely.Polygon(x + [x[0]])))

In [ ]:
ways['lon'] = ways.average_coord.apply(lambda x: x.y)

In [ ]:
ways['lat'] = ways.average_coord.apply(lambda y: y.x)

In [ ]:
ways = ways.drop(['refs', 'average_coord', 'coords'], axis=1)

In [ ]:
nodes = nodes.drop('id', axis = 1)
values_to_remove = ['city', 'town', 'suburb', 'village', 'hamlet']
nodes = nodes[~nodes['place'].isin(values_to_remove)]
nodes = nodes.drop('place', axis = 1)

In [ ]:
nodes = nodes.dropna()

In [ ]:
combined = pd.concat([ways, nodes], axis=0)

In [ ]:
combined.to_parquet(path='combined.parquet')

In [ ]:
query_string = f"""
COPY (
    WITH osm AS (
        SELECT
            name,
            lat,
            lon
        FROM 'combined.parquet'
        WHERE LEN(name) > 1
    ),
    overture AS (
        SELECT
            id,
            names.primary AS name,
            categories.primary AS category,
            geometry AS geometry
        FROM read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=places/*/*')
        WHERE id IN {gers_ids_list}
    )
    SELECT
        overture.id AS id,
        overture.name AS name,
        overture.category AS category,
        osm.lon AS lon,
        osm.lat AS lat
    FROM overture
        LEFT JOIN osm ON (
            osm.name = overture.name
            OR osm.name ilike concat('%', overture.name, '%')
            OR overture.name ilike concat('%', osm.name, '%')
            OR damerau_levenshtein(osm.name, overture.name) < 3
        )
        AND st_intersects(
            st_buffer(overture.geometry::geometry, 0.003),
            st_point(osm.lon, osm.lat)
        )
) TO 'ground_truth.parquet'
"""

%sql {{query_string}}

In [ ]:
ground_truth = pd.read_parquet(path='ground_truth.parquet')

In [ ]:
ground_truth = ground_truth.dropna()

In [ ]:
ground_truth = ground_truth.groupby('id').agg({
    'lon': 'mean',
    'lat': 'mean',
    'name': 'first',
    'category': 'first'
}).reset_index()

In [ ]:
ground_truth['original_coord'] = ground_truth.id.apply(lambda x: places[places['id'] == x]['geometry'].values[0])
ground_truth['snapped_coord'] = ground_truth.id.apply(lambda x: places[places['id'] == x]['geometry_updated'].values[0])

In [ ]:
ground_truth['osm_coord'] = list(zip(ground_truth.lon, ground_truth.lat))

In [ ]:
ground_truth['osm_coord'] = ground_truth.osm_coord.apply(lambda x: shapely.Point(x))

In [ ]:
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
ground_truth['original_projected'] = ground_truth.original_coord.apply(lambda point: Point(transformer.transform(point.x, point.y)))

In [ ]:
ground_truth['snapped_projected'] = ground_truth.snapped_coord.apply(lambda point: Point(transformer.transform(point.x, point.y)))

In [ ]:
ground_truth['osm_projected'] = ground_truth.osm_coord.apply(lambda point: Point(transformer.transform(point.x, point.y)))

In [ ]:
ground_truth['old_dist'] = ground_truth.apply(lambda x: shapely.distance(x['original_projected'], x['osm_projected']), axis=1)

In [ ]:
ground_truth['new_dist'] = ground_truth.apply(lambda x: shapely.distance(x['snapped_projected'], x['osm_projected']), axis=1)

In [ ]:
ground_truth['original_coord'] = ground_truth['original_coord'].apply(lambda x: x.wkt)
ground_truth['snapped_coord'] = ground_truth['snapped_coord'].apply(lambda x: x.wkt)
ground_truth['osm_coord'] = ground_truth['osm_coord'].apply(lambda x: x.wkt)
ground_truth = ground_truth.drop(['lon', 'lat', 'original_projected', 'snapped_projected', 'osm_projected'], axis=1)

ground_truth.to_csv(path_or_buf='merged_data.csv')
ground_truth.to_parquet(path='merged_data.parquet')